In [1]:
import numpy as np
import tensorflow as tf


In [2]:
def load_X(X_signals_paths):
    X_signals = []

    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
            ]]
        )
        file.close()

    return np.transpose(np.array(X_signals), (1, 2, 0))


# Load "y" (the neural network's training and testing outputs)

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()
    # Substract 1 to each output class for friendly 0-based indexing
    return y_ - 1

def one_hot(y_):
    """
    Function to encode output labels from number indexes.
    E.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    """
    y_ = y_.reshape(len(y_))
    n_values = int(np.max(y_)) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]

In [3]:
class Config(object):
    """
    define a class to store parameters,
    the input should be feature mat of training and testing
    Note: it would be more interesting to use a HyperOpt search space:
    https://github.com/hyperopt/hyperopt
    """

    def __init__(self, X_train, X_test):
        # Input data
        self.train_count = len(X_train)  # 7352 training series
        self.test_data_count = len(X_test)  # 2947 testing series
        self.n_steps = len(X_train[0])  # 128 time_steps per series

        # Training
        self.learning_rate = 0.00005
        self.lambda_loss_amount = 0.00005
        self.training_epochs = 500
        self.batch_size = 100
        
        learning_rate = 0.0025
        lambda_loss_amount = 0.0015
        training_iters = training_data_count * 300  # Loop 300 times on the dataset
        batch_size = 1500
        display_iter = 30000 

        # LSTM structure
        self.n_inputs = len(X_train[0][0])  # Features count is of 9: 3 * 3D sensors features over time
        self.n_hidden = 40  # nb of neurons inside the neural network
        self.n_classes = 2  # Final output classes
        self.W = {
            'hidden': tf.Variable(tf.random_normal([self.n_inputs, self.n_hidden])),
            'output': tf.Variable(tf.random_normal([self.n_hidden, self.n_classes]))
        }
        self.biases = {
            'hidden': tf.Variable(tf.random_normal([self.n_hidden], mean=1.0)),
            'output': tf.Variable(tf.random_normal([self.n_classes]))
        }

In [4]:
def LSTM_Network(_X, config):
    """Function returns a TensorFlow RNN with two stacked LSTM cells
    Two LSTM cells are stacked which adds deepness to the neural network.
    Note, some code of this notebook is inspired from an slightly different
    RNN architecture used on another dataset, some of the credits goes to
    "aymericdamien".
    Args:
        _X:     ndarray feature matrix, shape: [batch_size, time_steps, n_inputs]
        config: Config for the neural network.
    Returns:
        This is a description of what is returned.
    Raises:
        KeyError: Raises an exception.
      Args:
        feature_mat: ndarray fature matrix, shape=[batch_size,time_steps,n_inputs]
        config: class containing config of network
      return:
              : matrix  output shape [batch_size,n_classes]
    """
    # (NOTE: This step could be greatly optimised by shaping the dataset once
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, config.n_inputs])
    # new shape: (n_steps*batch_size, n_input)

    # Linear activation
    _X = tf.nn.relu(tf.matmul(_X, config.W['hidden']) + config.biases['hidden'])
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(_X, config.n_steps, 0)
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(config.n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(config.n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    # Get LSTM cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)

    # Get last time step's output feature for a "many to one" style classifier,
    # as in the image describing RNNs at the top of this page
    lstm_last_output = outputs[-1]

    # Linear activation
    return tf.matmul(lstm_last_output, config.W['output']) + config.biases['output']


In [5]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
if __name__ == "__main__":
    # -----------------------------
    # Step 1: load and prepare data
    # -----------------------------

    # Those are separate normalised input features for the neural network
    INPUT_SIGNAL_TYPES = [
        "body_acc_x_",
        "body_acc_y_",
        "body_acc_z_",
        "body_gyro_x_",
        "body_gyro_y_",
        "body_gyro_z_",
        "total_acc_x_",
        "total_acc_y_",
        "total_acc_z_"
    ]

    # Output classes to learn how to classify
    LABELS = [
        "SITTING",
        "STANDING",
    ]
    files=['sess002','sess003','sess004','sess005','sess006','sess007','sess008','sess009','sess010','sess011','sess012','sess013','sess014','sess015','sess016','sess017','sess018','sess019','sess020']
    values=[0,1,1,0,1,0,1,1,1,0,0,1,1,1,0,1,0,1,0,0]
    j=0
    #Loadind data
    n_features=6
    features = ['head_velocity_x','head_velocity_y','head_velocity_z','head_ang_velocity_x','head_ang_velocity_y','head_ang_velocity_z']
    window=10
    
    X = pd.read_csv('OculusData/'+'sess001' ,names=['time','head_position_x','head_position_y','head_position_z','head_velocity_x','head_velocity_y','head_velocity_z','head_acc_x','head_acc_y','head_acc_z','head_orientation_x','head_orientation_y','head_orientation_z','head_orientation_w','head_ang_velocity_x','head_ang_velocity_y','head_ang_velocity_z','head_ang_acc_x','head_ang_acc_y','head_ang_acc_z','left_hand_position_x','left_hand_position_y','left_hand_position_z','left_hand_velocity_x','left_hand_velocity_y','left_hand_velocity_z','left_hand_acc_x','left_hand_acc_y','left_hand_acc_z','left_hand_orientation_x','left_hand_orientation_y','left_hand_orientation_z','left_hand_orientation_w','left_hand_ang_velocity_x','left_hand_ang_velocity_y','left_hand_ang_velocity_z','left_hand_ang_acc_x','left_hand_ang_acc_y','left_hand_ang_acc_z','right_hand_position_x','right_hand_position_y','right_hand_position_z','right_hand_velocity_x','right_hand_velocity_y','right_hand_velocity_z','right_hand_acc_x','right_hand_acc_y','right_hand_acc_z','right_hand_orientation_x','right_hand_orientation_y','right_hand_orientation_z','right_hand_orientation_w','right_hand_ang_velocity_x','right_hand_ang_velocity_y','right_hand_ang_velocity_z','right_hand_ang_acc_x','right_hand_ang_acc_y','right_hand_ang_acc_z'])
    X=X[features]
    data_points=X.shape[0]//window
    #print(data_points)
    X=X.head(data_points*window)
    X=np.array(X)
    X=np.reshape(X,((data_points,window,n_features)))
    #print(X.shape)
    Y=np.zeros((data_points,2))
    Y[:,values[j]]=1
    j+=1
    
    for i in files:
        X0 = pd.read_csv('OculusData/'+i ,names=['time','head_position_x','head_position_y','head_position_z','head_velocity_x','head_velocity_y','head_velocity_z','head_acc_x','head_acc_y','head_acc_z','head_orientation_x','head_orientation_y','head_orientation_z','head_orientation_w','head_ang_velocity_x','head_ang_velocity_y','head_ang_velocity_z','head_ang_acc_x','head_ang_acc_y','head_ang_acc_z','left_hand_position_x','left_hand_position_y','left_hand_position_z','left_hand_velocity_x','left_hand_velocity_y','left_hand_velocity_z','left_hand_acc_x','left_hand_acc_y','left_hand_acc_z','left_hand_orientation_x','left_hand_orientation_y','left_hand_orientation_z','left_hand_orientation_w','left_hand_ang_velocity_x','left_hand_ang_velocity_y','left_hand_ang_velocity_z','left_hand_ang_acc_x','left_hand_ang_acc_y','left_hand_ang_acc_z','right_hand_position_x','right_hand_position_y','right_hand_position_z','right_hand_velocity_x','right_hand_velocity_y','right_hand_velocity_z','right_hand_acc_x','right_hand_acc_y','right_hand_acc_z','right_hand_orientation_x','right_hand_orientation_y','right_hand_orientation_z','right_hand_orientation_w','right_hand_ang_velocity_x','right_hand_ang_velocity_y','right_hand_ang_velocity_z','right_hand_ang_acc_x','right_hand_ang_acc_y','right_hand_ang_acc_z'])
        X0=X0[features]
        data_points=X0.shape[0]//window
        #print(data_points)
        X0=X0.head(data_points*window)
        X0=np.array(X0)
        X0=np.reshape(X0,((data_points,window,n_features)))
        #print(X0.shape)
        X=np.concatenate((X,X0))
        Y0=np.zeros((data_points,2))
        Y0[:,values[j]]=1
        Y=np.concatenate((Y,Y0))
        j+=1
    # print(X[0])
    #X_min = X.min(axis=1, keepdims=True)
    #X_max = X.max(axis=1, keepdims=True)
    #X=(X - X_min)/(X_max - X_min)
    #print(X[0])
    #print(X[0])
    
    
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)
    print(X_train.shape)
    
    #print(y_train.head(100))
    #print(Config(X_train, X_test))
    config = Config(X_train, X_test)
    print("Some useful info to get an insight on dataset's shape and normalisation:")
    print("features shape, labels shape, each features mean, each features standard deviation")
    print(X_test.shape, y_test.shape,
          np.mean(X_test), np.std(X_test))
    print("the dataset is therefore properly normalised, as expected.")
    print(config.n_classes)

(2614, 100, 6)
Some useful info to get an insight on dataset's shape and normalisation:
features shape, labels shape, each features mean, each features standard deviation
(1288, 100, 6) (1288, 2) -0.00306312191641 0.424876835798
the dataset is therefore properly normalised, as expected.
2


In [6]:
    X = tf.placeholder(tf.float32, [None, config.n_steps, config.n_inputs])
    Y = tf.placeholder(tf.float32, [None, config.n_classes])

    pred_Y = LSTM_Network(X, config)

    # Loss,optimizer,evaluation
    l2 = config.lambda_loss_amount * \
        sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
    # Softmax loss and L2
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=pred_Y)) + l2
    optimizer = tf.train.AdamOptimizer(
        learning_rate=config.learning_rate).minimize(cost)

    correct_pred = tf.equal(tf.argmax(pred_Y, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32))

In [7]:
    sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=False))
    init = tf.global_variables_initializer()
    sess.run(init)
    y_pre=y_test[0]
    best_accuracy = 0.0
    # Start training for each batch and loop epochs
    for i in range(config.training_epochs):
        #print(config.learning_rate)
        
        for start, end in zip(range(0, config.train_count, config.batch_size),
                              range(config.batch_size, config.train_count + 1, config.batch_size)):
            sess.run(optimizer, feed_dict={X: X_train[start:end],
                                           Y: y_train[start:end]})

        # Test completely at every epoch: calculate accuracy
        pred_out, accuracy_out, loss_out = sess.run(
            [pred_Y, accuracy, cost],
            feed_dict={
                X: X_test,
                Y: y_test
            }
        )
        print("traing iter: {},".format(i) +
              " test accuracy : {},".format(accuracy_out) +
              " loss : {}".format(loss_out))
        if accuracy_out>best_accuracy:
            y_pre=pred_out
            best_accurcy=accuracy_out

    print("")
    print("final test accuracy: {}".format(accuracy_out))
    print("best epoch's test accuracy: {}".format(best_accuracy))
    print("")   
    best_accuracy = max(best_accuracy, accuracy_out)


traing iter: 0, test accuracy : 0.4433229863643646, loss : 0.8223296403884888
traing iter: 1, test accuracy : 0.5077639818191528, loss : 0.753078281879425
traing iter: 2, test accuracy : 0.5015528202056885, loss : 0.7263519763946533
traing iter: 3, test accuracy : 0.5295031070709229, loss : 0.7091094255447388
traing iter: 4, test accuracy : 0.5318322777748108, loss : 0.7012753486633301
traing iter: 5, test accuracy : 0.5395962595939636, loss : 0.6963726878166199
traing iter: 6, test accuracy : 0.5582298040390015, loss : 0.6931930184364319
traing iter: 7, test accuracy : 0.5753105878829956, loss : 0.6908589601516724
traing iter: 8, test accuracy : 0.5838509202003479, loss : 0.6890290975570679
traing iter: 9, test accuracy : 0.5908384919166565, loss : 0.6874772906303406
traing iter: 10, test accuracy : 0.5931677222251892, loss : 0.6860877275466919
traing iter: 11, test accuracy : 0.590062141418457, loss : 0.684810996055603
traing iter: 12, test accuracy : 0.5877329111099243, loss : 0.683

traing iter: 105, test accuracy : 0.6420807242393494, loss : 0.6387128829956055
traing iter: 106, test accuracy : 0.6444099545478821, loss : 0.6386514902114868
traing iter: 107, test accuracy : 0.6451863646507263, loss : 0.638603687286377
traing iter: 108, test accuracy : 0.6451863646507263, loss : 0.6385706663131714
traing iter: 109, test accuracy : 0.6459627151489258, loss : 0.6385514736175537
traing iter: 110, test accuracy : 0.6459627151489258, loss : 0.6385458111763
traing iter: 111, test accuracy : 0.6451863646507263, loss : 0.6385535597801208
traing iter: 112, test accuracy : 0.6482919454574585, loss : 0.6385743021965027
traing iter: 113, test accuracy : 0.6498447060585022, loss : 0.6386072635650635
traing iter: 114, test accuracy : 0.6506211161613464, loss : 0.6386510729789734
traing iter: 115, test accuracy : 0.6498447060585022, loss : 0.6387055516242981
traing iter: 116, test accuracy : 0.6498447060585022, loss : 0.6387689113616943
traing iter: 117, test accuracy : 0.64906829

traing iter: 208, test accuracy : 0.6498447060585022, loss : 0.6553493738174438
traing iter: 209, test accuracy : 0.6498447060585022, loss : 0.6555939316749573
traing iter: 210, test accuracy : 0.6498447060585022, loss : 0.6558412909507751
traing iter: 211, test accuracy : 0.6506211161613464, loss : 0.6560911536216736
traing iter: 212, test accuracy : 0.6506211161613464, loss : 0.6563428640365601
traing iter: 213, test accuracy : 0.6506211161613464, loss : 0.6565978527069092
traing iter: 214, test accuracy : 0.6529502868652344, loss : 0.6568556427955627
traing iter: 215, test accuracy : 0.6513975262641907, loss : 0.6571162939071655
traing iter: 216, test accuracy : 0.6506211161613464, loss : 0.6573798060417175
traing iter: 217, test accuracy : 0.649068295955658, loss : 0.6576457023620605
traing iter: 218, test accuracy : 0.6498447060585022, loss : 0.6579147577285767
traing iter: 219, test accuracy : 0.6506211161613464, loss : 0.6581866145133972
traing iter: 220, test accuracy : 0.65062

KeyboardInterrupt: 

In [ ]:
b = np.zeros_like(y_pre)
b[np.arange(len(y_pre)), y_pre.argmax(1)] = 1
y_pre=b
#print(y_pre)
from sklearn.metrics import confusion_matrix
for i in range(y_test.shape[1]):
    print("Col {}".format(i))
    print(confusion_matrix(y_test[:,i], y_pre[:,i]))
    print("")

In [ ]:
n_classes=2

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
predictions = y_pre
%matplotlib inline
print("Testing Accuracy: {}%".format(100*accuracy))

print("")
print("Precision: {}%".format(100*metrics.precision_score(y_test, predictions, average="weighted")))
print("Recall: {}%".format(100*metrics.recall_score(y_test, predictions, average="weighted")))
print("f1_score: {}%".format(100*metrics.f1_score(y_test, predictions, average="weighted")))

print("")
print("Confusion Matrix:")
confusion_matrix = metrics.confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))
print(confusion_matrix)
normalised_confusion_matrix = np.array(confusion_matrix, dtype=np.float32)/np.sum(confusion_matrix)*100

print("")
print("Confusion matrix (normalised to % of total test data):")
print(normalised_confusion_matrix)
print("Note: training and testing data is not equally distributed amongst classes, ")
print("so it is normal that more than a 6th of the data is correctly classifier in the last category.")

# Plot Results: 
width = 12
height = 12
plt.figure(figsize=(width, height))
plt.imshow(
    normalised_confusion_matrix, 
    interpolation='nearest', 
    cmap=plt.cm.rainbow
)
plt.title("Confusion matrix \n(normalised to % of total test data)")
plt.colorbar()
tick_marks = np.arange(n_classes)
plt.xticks(tick_marks, LABELS, rotation=90)
plt.yticks(tick_marks, LABELS)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()